In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import Levenshtein
import time
import torchaudio
import librosa


from morse.my_datasets import read_dataset_from_files, generate_dataset
from morse.augmentations import make_mel_feature_extraction_transform, normalize_mel_spec, make_compose_transform, make_noise_signal_transform, make_runtime_mel_bounded_noise_transform, make_volume_signal_transform, make_runtime_rotation_transform, make_mel_tail_zeroing_transform

In [2]:
labels_dir = '../'
audio_dir = '../morse_dataset'


# dev_flag = True


full_train_df = pd.read_csv(Path(labels_dir, 'train.csv'))
test_df = pd.read_csv(Path(labels_dir, 'test.csv'))
full_train_df.head()

,id,message
0,1.opus,03ЩУЫЛПИГХ
1,2.opus,ЪЛТ0ДС6А3Г
2,3.opus,5ЭКЫБЗХЯН
3,4.opus,ЖЫЦОИ68КФ
4,5.opus,32Ю7МЫ ЗЛ


In [3]:
from sklearn.model_selection import train_test_split

train_index, val_index = train_test_split(np.arange(full_train_df.shape[0]), test_size=1/6, shuffle=True, 
                                           random_state=42)

val_index = val_index[:100]
train_index = train_index[:100]

real_val_set = read_dataset_from_files(audio_dir, 
                                       filenames = full_train_df.iloc[val_index]['id'], 
                                       labels=list(full_train_df.iloc[val_index]['message']), 
                                    #    mel_spec_transform=make_mel_feature_extraction_transform(4)
                                       )
print(len(real_val_set))

real_train_set = read_dataset_from_files(audio_dir, 
                                       filenames = full_train_df.iloc[train_index]['id'], 
                                       labels=list(full_train_df.iloc[train_index]['message']),
                                    #    mel_spec_transform=make_mel_feature_extraction_transform(4)
                                       )
print(len(real_train_set))

100%|██████████| 100/100 [00:03<00:00, 28.71it/s]


100


100%|██████████| 100/100 [00:01<00:00, 80.58it/s]

100


In [4]:
test_set = read_dataset_from_files(audio_dir, 
                                       filenames = list(test_df['id'])[:100], 
                                       labels=['' for i in range(5000)][:100], 
                                    #    mel_spec_transform=make_mel_feature_extraction_transform(4)
                                       )
print(len(test_set))

100%|██████████| 100/100 [00:01<00:00, 70.64it/s]

100


In [5]:
volume_tr_min_res=0.2
volume_tr_max_freq=0.5
noise_signal_tr__max_volume = 1.83
# noise_signal_tr__prob = 0.6

# bounded_noise_tr__prob = 0.96
bounded_noise_tr__max_volume = 0.6
bounded_noise_tr__std_frac_max_bound = 0.12

inner_dot_duration_multiplier_deviation = 0.1

runtime_rotation_transform__prob = 0.1

signal_tr = make_compose_transform([
    make_volume_signal_transform(min_res=volume_tr_min_res, max_freq=volume_tr_max_freq),
    make_noise_signal_transform(max_volume=noise_signal_tr__max_volume, p=42),
])

inner_dot_duration_multiplier_range=(1 - inner_dot_duration_multiplier_deviation, 1 + inner_dot_duration_multiplier_deviation)



fake_set = generate_dataset(100, signal_transform=signal_tr, mel_spec_transform=make_compose_transform([make_mel_tail_zeroing_transform(),
                                                                                                    #    make_mel_feature_extraction_transform(4),
                                                                                                       normalize_mel_spec]),
                                  inner_dot_duration_multiplier_range=inner_dot_duration_multiplier_range, show_pbar=True)
# fake_val_set = generate_dataset(fake_val_set_size, signal_transform=signal_tr, mel_spec_transform=bounded_noise_tr,
#                                  inner_dot_duration_multiplier_range=inner_dot_duration_multiplier_range, show_pbar=False)

100%|██████████| 100/100 [00:00<00:00, 139.01it/s]


In [18]:
# for i in range(100):
#     train_mel = real_train_set[i][0]
#     val_mel = real_val_set[i][0]
#     test_mel = test_set[i][0]
#     fake_mel = fake_set[i][0]
#     fig, (ax1, ax2, ax3, ax4) = plt.subplots(4)
#     ax1.imshow(train_mel, aspect=2)
#     ax2.imshow(val_mel, aspect=2)
#     ax3.imshow(test_mel, aspect=2)
#     ax4.imshow(fake_mel, aspect=2)
#     plt.show()